In [1]:
import persistqueue

In [2]:
q = persistqueue.SQLiteAckQueue('./my_queue')

In [3]:
q.put("str 0")
q.put("str 1")
q.put("str 2")
q.put("str 3")
q.put("str 4")
q.put("str 5")
q.put("str 6")

In [4]:
q.size

7

In [12]:
a = q.get()

In [13]:
a

'str 1'

In [14]:
q.size

5

In [15]:
q.nack(a)

In [16]:
q.size

6